# Import

We are importing the neccessary components that we are going to use in the notebook.

- Keras backend with tensorflow
- Preprossing
    - Tokenisation
    - Removing stopwords
- GRU Networks
- Attention Mechanism



In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

import helper
import collections
import tests
import numpy as np

# Devices

Devices are technically processors that are present in your computer. It could be 

- GPU ( Graphical Processing Unit )
- CPU


In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4613344301778406225
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8383517297127865851
physical_device_desc: "device: XLA_CPU device"
]


# Dataset

- Importing the dataset (small_vocab_en) file containing the Language English
- Importing the dataset (small_vocab_fr) file containing the Language French

In [16]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


In [17]:
for sample_i in range(5):
    print('French sample {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))
    print('English sample {}:  {}\n'.format(sample_i + 1, english_sentences[sample_i]))

French sample 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
English sample 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .

French sample 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .
English sample 2:  the united states is usually chilly during july , and it is usually freezing in november .

French sample 3:  california est généralement calme en mars , et il est généralement chaud en juin .
English sample 3:  california is usually quiet during march , and it is usually hot in june .

French sample 4:  les états-unis est parfois légère en juin , et il fait froid en septembre .
English sample 4:  the united states is sometimes mild during june , and it is cold in september .

French sample 5:  votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .
English sample 5:  your least liked fruit is the grape , but my least liked is the apple .



In [9]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])


print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')
print()
print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')


1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"


In [10]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'TLe renard brun saute par dessus le chien paresseux',
    'Par Jove , mon étude rapide de la lexicographie a remporté un prix .',
    'C\'est une courte phrase .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'par': 1, 'tle': 2, 'renard': 3, 'brun': 4, 'saute': 5, 'dessus': 6, 'le': 7, 'chien': 8, 'paresseux': 9, 'jove': 10, 'mon': 11, 'étude': 12, 'rapide': 13, 'de': 14, 'la': 15, 'lexicographie': 16, 'a': 17, 'remporté': 18, 'un': 19, 'prix': 20, "c'est": 21, 'une': 22, 'courte': 23, 'phrase': 24}

Sequence 1 in x
  Input:  TLe renard brun saute par dessus le chien paresseux
  Output: [2, 3, 4, 5, 1, 6, 7, 8, 9]
Sequence 2 in x
  Input:  Par Jove , mon étude rapide de la lexicographie a remporté un prix .
  Output: [1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Sequence 3 in x
  Input:  C'est une courte phrase .
  Output: [21, 22, 23, 24]


In [11]:

def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return pad_sequences(x, maxlen=length, padding='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [2 3 4 5 1 6 7 8 9]
  Output: [2 3 4 5 1 6 7 8 9 0 0 0]
Sequence 2 in x
  Input:  [ 1 10 11 12 13 14 15 16 17 18 19 20]
  Output: [ 1 10 11 12 13 14 15 16 17 18 19 20]
Sequence 3 in x
  Input:  [21 22 23 24]
  Output: [21 22 23 24  0  0  0  0  0  0  0  0]


In [12]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [13]:

def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [15]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = Sequential()
    model.add(GRU(256, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    # Compile model
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

#tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_french_sentences, max_english_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[-1], 1))

shape = preproc_english_sentences.shape

preproc_english_sentences = preproc_english_sentences.reshape( ( -1 , shape[-1] , 1 ) )

print(preproc_english_sentences.shape)
print(tmp_x.shape)
print(max_french_sequence_length)
print(english_vocab_size)
print(french_vocab_size)

# Train the neural network
simple_rnn_model = simple_model( 
    tmp_x.shape,
    max_english_sequence_length,
    french_vocab_size,
    english_vocab_size)

print(simple_rnn_model.summary())

simple_rnn_model.fit(tmp_x, preproc_english_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], english_tokenizer))


(137861, 15, 1)
(137861, 15, 1)
21
199
344
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_2 (GRU)                  (None, 15, 256)           198144    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 15, 1024)          263168    
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 1024)          0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 15, 199)           203975    
Total params: 665,287
Trainable params: 665,287
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
  1024/110288 [..........................

InvalidArgumentError: Received a label value of 199 which is outside the valid range of [0, 199).  Label values: 24 1 8 56 4 45 6 3 1 10 63 2 35 0 0 18 1 57 4 43 7 3 1 9 63 2 45 0 0 0 5 90 1 31 12 16 13 6 5 91 1 32 12 16 0 27 69 79 76 7 77 0 0 0 0 0 0 0 0 0 96 92 73 78 7 75 0 0 0 0 0 0 0 0 0 22 1 67 4 35 6 3 1 9 60 2 47 0 0 0 5 20 21 1 8 62 4 42 6 3 1 10 66 2 38 5 87 1 30 12 11 13 6 5 88 1 29 12 11 0 24 1 8 64 4 35 6 3 1 66 2 37 0 0 0 5 89 1 30 12 16 13 6 5 91 1 50 12 16 0 50 12 16 13 1 5 89 6 49 12 16 1 5 85 0 5 91 1 29 14 15 13 6 5 84 1 30 14 15 0 26 70 28 77 7 72 0 0 0 0 0 0 0 0 0 24 1 10 55 4 47 6 3 1 9 59 2 34 0 0 22 1 64 4 46 7 3 1 10 57 2 33 0 0 0 49 12 16 13 1 5 83 6 31 12 16 1 5 28 0 19 1 56 4 35 7 3 1 10 66 2 37 0 0 0 18 1 52 4 39 6 3 1 67 2 40 0 0 0 0 5 20 21 1 8 55 4 54 7 3 1 9 52 2 37 24 1 9 57 4 46 6 3 1 8 41 2 47 0 0 98 92 76 77 7 72 0 0 0 0 0 0 0 0 0 22 1 10 68 4 54 6 3 1 10 55 2 37 0 0 32 12 11 13 1 5 84 6 31 12 11 1 5 83 0 24 1 59 4 33 7 3 1 65 2 40 0 0 0 0 96 92 75 8